In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import math
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega
from pandas_datareader import data as pdr
from scipy.stats import norm, t
import seaborn as sns
import scipy
import statsmodels.api as sm
from dateutil.parser import parse
import scipy.optimize as sc

# Question 1

## Option Greeks

In [2]:
startDate = date(2022, 3, 13)
endDate = date(2022, 4, 15)
days = (endDate - startDate).days
tradingDays = 255
T = days / tradingDays #using trading days 

### Delta

In [3]:
def greek_delta (S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    if optionType == "Call":
        delta = norm.cdf(d1,0,1)
    else:
        delta = -norm.cdf(-d1,0,1)
        
    return delta

In [4]:
K = 165
S = 165
sigma = 0.2
r = 0.0025
couponRate = 0.0053
b = r - couponRate

delta_call = greek_delta(S, K, T, sigma, r, b, optionType = "Call")
delta_put = greek_delta(S, K, T, sigma, r, b, optionType = "Put")
print("The delta for call option is: %.4f" % delta_call)
print("The delta for put option is: %.4f" % delta_put)

The delta for call option is: 0.5123
The delta for put option is: -0.4877


### Gamma

In [5]:
def greek_gamma (S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    gamma = norm.pdf(d1, 0, 1) / (S*sigma*np.sqrt(T))
        
    return gamma

In [6]:
gamma_call = greek_gamma(S, K, T, sigma, r, b, optionType = "Call")
gamma_put = greek_gamma(S, K, T, sigma, r, b, optionType = "Put")
print("The gamma for call option is: %.4f" % gamma_call)
print("The gamma for put option is: %.4f" % gamma_put)

The gamma for call option is: 0.0336
The gamma for put option is: 0.0336


### Vega

In [7]:
def greek_vega (S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    vega = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
        
    return vega*0.01

In [8]:
vega_call = greek_vega(S, K, T, sigma, r, b, optionType = "Call")
vega_put = greek_vega(S, K, T, sigma, r, b, optionType = "Put")
print("The vega for call option is: %.4f" % vega_call)
print("The vega for put option is: %.4f" % vega_put)

The vega for call option is: 0.2367
The vega for put option is: 0.2367


### Theta

In [9]:
def greek_theta (S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if optionType == "Call":
        theta = -S * norm.pdf(d1,0,1) * sigma / (2*np.sqrt(T)) - r * K * np.exp(-r*T) * norm.cdf(d2,0,1)
    else:
        theta = -S * norm.pdf(d1,0,1) * sigma / (2*np.sqrt(T)) + r * K * np.exp(-r*T) * norm.cdf(-d2,0,1)
        
    return theta/365

In [10]:
theta_call = greek_theta(S, K, T, sigma, r, b, optionType = "Call")
theta_put = greek_theta(S, K, T, sigma, r, b, optionType = "Put")
print("The theta for call option is: %.4f" % theta_call)
print("The theta for put option is: %.4f" % theta_put)

The theta for call option is: -0.0507
The theta for put option is: -0.0495


### Rho

In [11]:
def greek_rho (S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if optionType == "Call":
        rho = K*T*np.exp(-r*T)*norm.cdf(d2,0,1)
    else:
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2,0,1)
        
    return rho*0.01

In [12]:
rho_call = greek_rho(S, K, T, sigma, r, b, optionType = "Call")
rho_put = greek_rho(S, K, T, sigma, r, b, optionType = "Put")
print("The rho for call option is: %.4f" % rho_call)
print("The rho for put option is: %.4f" % rho_put)

The rho for call option is: 0.1032
The rho for put option is: -0.1102


In [13]:
name = ["Delta", "Gamma", "Vega", "Theta", "Rho"]
call_value = [delta_call, gamma_call, vega_call, theta_call, rho_call]
put_value = [delta_put, gamma_put, vega_put, theta_put, rho_put]

result = pd.DataFrame(list(zip(name, call_value, put_value)), columns = ["Option Greeks", "Call Option Value", "Put Option Value"])
result

,Option Greeks,Call Option Value,Put Option Value
0,Delta,0.512340,-0.487660
1,Gamma,0.033589,0.033589
2,Vega,0.236686,0.236686
3,Theta,-0.050654,-0.049525
4,Rho,0.103239,-0.110222


## Binomial tree valuation for American option without discrete dividends

In [14]:
def american_without_div (K, startDate, endDate, S0, sigma, r, N, optionType):
    '''
    S0: Underlying Price
    K: Strike Price
    T: Time to Maturity by trading days in a year
    sigma: Implied Volatility
    r: Risk free rate
    N: number of time steps
    u: up-factor in binomial tree
    d: down-factor in binomial tree
    p: probability for price up
    disc: discount rate
    *format of Date: startDate = date(2022, 2, 25)
    '''
    
    days = (endDate - startDate).days
    tradingDays = 255
    T = days / tradingDays #using trading days 
    t = T/N
    disc = np.exp(-r*t)
    u = np.exp(sigma*np.sqrt(t))
    d = 1/u
    p = (np.exp(r*t) - d) / (u-d)
    
    #calculate the stock price at maturity
    S = np.zeros(N+1) #there are N+1 nodes at maturity (last column)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j)
        
    #calculate option payoff
    C = np.zeros(N+1)
    for j in range (0, N+1):
        if optionType == "Call":
            C[j] = max(0, S[j] - K)
        elif optionType == "Put":
            C[j] = max(0, K - S[j])
        else:
            raise TypeError("optionType must be Call or Put")
            
    #calculate the option value
    for i in np.arange(N-1,-1,-1):
        for j in range(0, i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = disc * ( p*C[j+1] + (1-p)*C[j] )
            if optionType == "Call":
                C[j] = max(C[j], S - K)
            elif optionType == "Put":
                C[j] = max(C[j], K - S)
            else:
                raise TypeError("optionType must be Call or Put")
                
    return C[0]

In [15]:
K = 165
startDate = date(2022, 3, 13)
endDate = date(2022, 4, 15)
S0 = 165
sigma = 0.2
r = 0.0025
couponRate = 0.0053
b = r - couponRate
N = 1

american_without_call = american_without_div(K, startDate, endDate, S0, sigma, r, N, optionType = "Call")
american_without_put = american_without_div(K, startDate, endDate, S0, sigma, r, N, optionType = "Put")
print("The call option value for American options without discrete dividends is: $%.4f" % american_without_call)
print("The put option value for American options without discrete dividends is: $%.4f" % american_without_put)

The call option value for American options without discrete dividends is: $5.9589
The put option value for American options without discrete dividends is: $5.9055


## Binomial tree valuation for American option with discrete dividends

![AmericanOption_with_dividend_calculate](./AmericanOption_with_dividend_calculate.jpg)

In [16]:
def american_with_div (K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType):
    '''
    S0: Underlying Price
    K: Strike Price
    sigma: Implied Volatility
    r: Risk free rate
    u: up-factor in binomial tree
    d: down-factor in binomial tree
    p: probability for price up
    div: dividend value
    *format of Date: startDate = date(2022, 2, 25)
    '''
    
    tradingDays = 255
    #calculate days from start to div, start to end, div to end
    start2div = (divDate - startDate).days
    div2end = (endDate - divDate).days
    start2end = (endDate - startDate).days
    
    #calculate the rate of each part using trading days 
    ttm = start2end / tradingDays
    ttm_start2div = start2div / tradingDays
    ttm_div2end = div2end / tradingDays
    
    #calculate u, d, and p
    u = np.exp(sigma*np.sqrt(ttm))
    d = 1/u
    p = (np.exp(r*ttm) - d) / (u-d)
    
    #calculate the stock price and option price at maturity
    D0 = div / (1+r)**ttm_start2div
    S0 = S0 - D0
    
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**(N-j) * d**j
        
    C = np.zeros(N+1)
    for j in range (0, N+1):
        if optionType == "Call":
            C[j] = max(0, S[j] - K)
        elif optionType == "Put":
            C[j] = max(0, K - S[j])
        else:
            raise TypeError("optionType must be Call or Put")
            
    #calculate the stock price at dividend period
    S1 = np.zeros(N)
    for j in range(0, N):
        S1[j] = S0 * u**(N-1-j) * d**j
    
    #calculate the option price executing at dividend period
    C1_exe = np.zeros(N)
    for j in range (0, N):
        if optionType == "Call":
            temp = max(0, S1[j] - K)
            if temp > 0:
                C1_exe[j] = temp + div
            else:
                C1_exe[j] = temp
        elif optionType == "Put":
            temp = max(0, K - S1[j])
            if temp > 0:
                C1_exe[j] = temp + div
            else:
                C1_exe[j] = temp
        else:
            raise TypeError("optionType must be Call or Put")
            
    #calculate the option price executing at maturity at dividend period
    C1 = np.zeros(N)
    for j in range(0, N):
        C1[j] = (p*C[j] + (1-p)*C[j+1]) / (1+r)**ttm_div2end
        
    #compare C1 and C1_exe to determined whether execute option at dividend period
    C1_max = np.zeros(N)
    for j in range(N):
        C1_max[j] = max(C1[j], C1_exe[j])
        
    #calculate the option value
    C0 = (p*C1_max[0] + (1-p)*C1_max[1]) / (1+r)**ttm_start2div
    
    return C0

In [17]:
K = 165
startDate = date(2022, 3, 13)
endDate = date(2022, 4, 15)
divDate = date(2022, 4, 11)
div = 0.88
S0 = 165
sigma = 0.2
r = 0.0025
couponRate = 0.0053
b = r - couponRate
N = 2

american_withDiv_call = american_with_div(K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType = "Call")
american_withDiv_put = american_with_div(K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType = "Put")
print("The call option value for American options with discrete dividends is: $%.4f" % american_withDiv_call)
print("The put option value for American options with discrete dividends is: $%.4f" % american_withDiv_put)

The call option value for American options with discrete dividends is: $5.9274
The put option value for American options with discrete dividends is: $7.0012


In [18]:
name = ["American option value without dividends", "American option value with dividends"]
call_value = [american_without_call, american_withDiv_call]
put_value = [american_without_put, american_withDiv_put]

result = pd.DataFrame(list(zip(name, call_value, put_value)), columns = ["American option price with/without discrete dividends", "Call Option Value", "Put Option Value"])
result

,American option price with/without discrete dividends,Call Option Value,Put Option Value
0,American option value without dividends,5.958853,5.905479
1,American option value with dividends,5.927437,7.001176


# Question 2

## Import some functions

In [19]:
def impliedVolatility (S, X, T, r, market_price, tol, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    r: Risk free rate
    tol: tolerance
    '''
    
    max_iter = 9999
    iv_old = 0.5
    
    for i in range(max_iter):
        bs_price = bs(optionType, S, X, T, r, iv_old)
        vega_price = vega(optionType, S, X, T, r, iv_old) * 100
        gap = bs_price - market_price
        
        iv_new = iv_old - gap/vega_price
        new_bs_price = bs(optionType, S, X, T, r, iv_new)
        
        if (abs(iv_old - iv_new) < tol or abs(new_bs_price - bs_price) < tol):
            break
        
        iv_old = iv_new
        
    impliedVolatility = iv_new
    return impliedVolatility

In [20]:
def AmericanValue(port, underlying_values, simulated_date):
    result = pd.DataFrame(port['Portfolio'])
    
    for price in underlying_values:
        values = []
        for i in range(len(port)):
            if port['Type'][i] == 'Option':
                if port['OptionType'][i] == 'Call': #value = american_with_div (K = port["Strike"][i], startDate, endDate, divDate, div, S0 = price, sigma = port["ImpliedVolatility"][i], r, N, optionType = "Call")
                    value = american_with_div ( port["Strike"][i], startDate, endDate, divDate, div, price, port["ImpliedVolatility"][i], r, N, "Call")
                else:
                    value = american_with_div (port["Strike"][i], startDate, endDate, divDate, div, price, port["ImpliedVolatility"][i], r, N, "Put")
                values.append(value)
            else:
                values.append(price)
        result[str(price)] = np.array(values)
        
    result = result.groupby('Portfolio').sum()
    return result

In [21]:
def parametricVaR (mean, std, distribution, alpha, dof):
    if distribution == "normal":
        VaR = (1-alpha/100) * std - mean #since normal distribution is symmetric, it is easy to calculate by using 1-alpha/100
    elif distribution == "t-distribution":
        VaR = np.sqrt((dof-2)/dof) * t.ppf(1-alpha/100, dof) * std - mean
    else:
        raise TypeError("Expected the distribution to be normal or t-distribution.")
    return VaR

In [22]:
def parametricCVaR (mean, std, distribution, alpha, dof):
    if distribution == "normal":
        CVaR = (alpha/100)**-1 * norm.pdf( norm.ppf(alpha/100) ) * std - mean
    elif distribution == "t-distribution":
        x = t.ppf(alpha/100, dof)
        CVaR = -1/(alpha/100) * (1-dof)**-1 * (dof-2+x**2) * t.pdf(x,dof) *std - mean
    else:
        raise TypeError("Expected the distribution to be normal or t-distribution.")
    return CVaR

## Add Implied Volatility to portfolio 

In [23]:
#load data
port = pd.read_csv("problem2.csv")
ret = pd.read_csv("DailyReturn.csv", index_col = 0)['AAPL']

In [24]:
#calculate ttm
tradingDays = 255

startDate = date(2022, 2, 25)
endDate = date(2022, 3, 18)
start2end = (endDate - startDate).days

ttm = start2end / tradingDays

In [25]:
# reset some value for impliedVolatility function
S = 164.85
T = ttm
r = 0.0025
couponRate = 0.0053
b = r - couponRate
tol = 0.00001

In [26]:
# add IV to port
IV = []

for i in range(len(port.index)):
    if port.Type[i] == "Stock":
        IV.append(None)
    else:
        if port.OptionType[i] == "Call":
            iv = impliedVolatility(S, port.Strike[i], T, r, port.CurrentPrice[i], tol, "c")
            IV.append(iv)
        else:
            iv = impliedVolatility(S, port.Strike[i], T, r, port.CurrentPrice[i], tol, "p")
            IV.append(iv)
    
port["ImpliedVolatility"] = IV
port

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,ImpliedVolatility
0,Straddle,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.241462
1,Straddle,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.230012
2,SynLong,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.241462
3,SynLong,Option,AAPL,-1,Put,3/18/2022,165.0,4.40,0.230012
4,CallSpread,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.241462
5,CallSpread,Option,AAPL,-1,Call,3/18/2022,175.0,0.72,0.196834
6,PutSpread,Option,AAPL,1,Put,3/18/2022,165.0,4.40,0.230012
7,PutSpread,Option,AAPL,-1,Put,3/18/2022,155.0,1.60,0.277916
8,Stock,Stock,AAPL,1,NaN,NaN,NaN,164.85,NaN
9,Call,Option,AAPL,1,Call,3/18/2022,165.0,4.50,0.241462


## Simulate price for each portfolio

In [27]:
# reset some value
startDate = date(2022, 2, 25)
endDate = date(2022, 3, 18)
divDate = date(2022, 3, 15)
div = 1
r = 0.0025
N=2

In [28]:
port['Value'] = port["CurrentPrice"]*port['Holding']
portval = port.groupby('Portfolio').sum()['Value']

In [29]:
std = ret.std()
nsample = 5000
rets_sim = scipy.stats.norm(0, std).rvs((10, nsample))
price_sim = pd.DataFrame(164.85 * (1 + rets_sim).prod(axis=0))

portval_bar = AmericanValue(port, price_sim.values,date(2022,3,7))
port_list = (portval_bar.div(portval,axis=0)-1).T


<ipython-input-20-40b886d3089c>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result[str(price)] = np.array(values)


In [30]:
port_list

Portfolio,Call,CallSpread,CoveredCall,ProtectedPut,Put,PutSpread,Stock,Straddle,SynLong
[163.16215836],0.107660,0.731152,0.048623,0.011400,0.822125,3.471776,-0.010239,0.460879,129.018221
[167.55316335],0.646397,1.681790,0.091125,0.020653,0.180096,2.103313,0.016398,0.415866,125.012099
[161.67892589],0.018927,0.521045,0.036882,0.009049,1.068775,3.980825,-0.019236,0.537953,135.877820
[181.52209598],3.032024,6.359591,0.245190,0.083165,-0.590100,-0.250852,0.101135,1.241311,198.476664
[157.64893965],-0.222164,-0.049825,0.004984,0.002660,1.738930,5.407684,-0.043683,0.747366,154.515550
...,...,...,...,...,...,...,...,...,...
[163.36167648],0.119596,0.759415,0.050202,0.011717,0.788947,3.403301,-0.009028,0.450511,128.095497
[168.2070434],0.758067,1.860788,0.098337,0.022771,0.112934,1.944229,0.020364,0.439125,127.082134
[171.63266267],1.343098,2.798539,0.136118,0.038309,-0.067938,1.379491,0.041144,0.645507,145.450103
[161.50061553],0.008260,0.495786,0.035471,0.008766,1.098427,4.042022,-0.020318,0.547219,136.702463


## Calculate Empirical VaR and CVaR

In [31]:
alpha = 5
dof = nsample - 1

name = []
Mean, VaR, CVaR, VaR_val, CVaR_val = [], [], [], [], []
for strategy in port_list.columns:
    data = port_list[strategy]
    
    Mean.append(data.mean())
    
    empVaR = np.percentile(data, alpha)
    belowVaR = data <= empVaR
    empCVaR = data[belowVaR].mean()
    
    empVaR_val = port_list[strategy].sum() * empVaR * np.sqrt(start2end)
    empCVaR_val = port_list[strategy].sum() * empCVaR * np.sqrt(start2end)
    
    VaR.append(empVaR)
    CVaR.append(empCVaR)
    VaR_val.append(empVaR_val)
    CVaR_val.append(empCVaR_val)
    
    name.append(strategy)

result = pd.DataFrame(list(zip(name, Mean, VaR, CVaR, VaR_val, CVaR_val)), columns = ["Strategy", "Mean", "Empirical VaR", "Empirical CVaR", "Empirical VaR Value ($)", "Empirical CVaR Value ($)"])
result

,Strategy,Mean,Empirical VaR,Empirical CVaR,Empirical VaR Value ($),Empirical CVaR Value ($)
0,Call,0.569366,-0.593778,-0.772838,-7.746321e+03,-1.008230e+04
1,CallSpread,1.520234,-0.516402,-0.729569,-1.798782e+04,-2.541303e+04
2,CoveredCall,0.071940,-0.044183,-0.069911,-7.282982e+01,-1.152377e+02
3,ProtectedPut,0.021147,-0.007187,-0.011068,-3.482632e+00,-5.363111e+00
4,Put,0.819493,-0.438200,-0.651071,-8.228047e+03,-1.222512e+04
5,PutSpread,3.477461,0.223425,-0.336937,1.780222e+04,-2.684665e+04
6,Stock,-0.000161,-0.081364,-0.101501,3.005488e-01,3.749336e-01
7,Straddle,0.693024,0.379286,0.372344,6.022755e+03,5.912511e+03
8,SynLong,149.679150,121.756480,121.138584,4.175736e+08,4.154545e+08


## Calculate VaR and CVaR using Delta-normal

In [32]:
alpha = 5
dof = nsample - 1
distribution = "normal"

name = []
Mean, deltaVaR, deltaCVaR, deltaVaR_val, deltaCVaR_val = [], [], [], [], []
for strategy in port_list.columns:
    data = port_list[strategy]
    
    Mean.append(data.mean())
    
    delta_VaR = parametricVaR (data.mean(), data.std(), distribution, alpha, dof)
    delta_CVaR = parametricCVaR (data.mean(), data.std(), distribution, alpha, dof)
    
    delta_VaR_val = port_list[strategy].sum() * delta_VaR * np.sqrt(start2end)
    delta_CVaR_val = port_list[strategy].sum() * delta_CVaR * np.sqrt(start2end)
    
    deltaVaR.append(delta_VaR)
    deltaCVaR.append(delta_CVaR)
    deltaVaR_val.append(delta_VaR_val)
    deltaCVaR_val.append(delta_CVaR_val)
    
    name.append(strategy)

result = pd.DataFrame(list(zip(name, Mean, deltaVaR, deltaCVaR, deltaVaR_val, deltaCVaR_val)), columns = ["Strategy", "Mean", "Delta VaR", "Delta CVaR", "Delta VaR Value ($)", "Delta CVaR Value ($)"])
result

,Strategy,Mean,Delta VaR,Delta CVaR,Delta VaR Value ($),Delta CVaR Value ($)
0,Call,0.569366,0.393118,1.520454,5.128551e+03,1.983557e+04
1,CallSpread,1.520234,0.275525,2.378856,9.597347e+03,8.286256e+04
2,CoveredCall,0.071940,0.004128,0.093226,6.805184e+00,1.536686e+02
3,ProtectedPut,0.021147,0.002195,0.029536,1.063822e+00,1.431179e+01
4,Put,0.819493,0.179765,1.350172,3.375436e+03,2.535209e+04
5,PutSpread,3.477461,-0.996889,1.908547,-7.943081e+04,1.520704e+05
6,Stock,-0.000161,0.048577,0.105286,-1.794396e-01,-3.889155e-01
7,Straddle,0.693024,-0.419398,-0.098906,-6.659697e+03,-1.570551e+03
8,SynLong,149.679150,-125.326431,-96.802660,-4.298170e+08,-3.319925e+08


# Question 3

## Preparation for the data

In [33]:
#load database
ret = pd.read_csv("DailyReturn.csv")
ff = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
mom = pd.read_csv("F-F_Momentum_Factor_daily.csv")

In [34]:
#merge the ff and mom
data = pd.merge(ff, mom)

#transfer percentage to value for all factors
data[["Mkt-RF", "SMB", "HML", "RF", "Mom"]] = data[["Mkt-RF", "SMB", "HML", "RF", "Mom"]] / 100

data

,Date,Mkt-RF,SMB,HML,RF,Mom
0,19261103,0.0020,-0.0022,-0.0029,0.00013,0.0056
1,19261104,0.0059,-0.0014,0.0070,0.00013,-0.0050
2,19261105,0.0007,-0.0008,0.0025,0.00013,0.0117
3,19261106,0.0016,-0.0029,0.0005,0.00013,-0.0003
4,19261108,0.0052,-0.0007,0.0008,0.00013,-0.0001
...,...,...,...,...,...,...
25062,20220125,-0.0143,-0.0060,0.0279,0.00000,0.0026
25063,20220126,-0.0030,-0.0108,0.0009,0.00000,0.0139
25064,20220127,-0.0078,-0.0163,0.0077,0.00000,0.0030
25065,20220128,0.0245,-0.0015,-0.0207,0.00000,-0.0027


### Change daily return date to int64 type

In [35]:
#change the ret Date to int64 format
#the date format must be month/day/year and transfer to year_month_day
# eg. (10/27/2022) to 20221027

def Date2int64 (vector):
    n = len(vector)
    dateInt = np.zeros(n)
    
    for i in range(n):
        date = vector[i]
        data_list = date.split("/")
        
        temp = []
        temp.append(data_list[2])
        temp.append(data_list[0])
        temp.append(data_list[1])
        
        dateInt[i] = int("".join(temp))
        
    return dateInt

In [36]:
ret["Date"] = Date2int64(ret["Date"])

### Find the rows with same date in daily return dateframe and in factors dateframe

In [37]:
data_ret = pd.merge(data, ret)
data_ret

,Date,Mkt-RF,SMB,HML,RF,Mom,SPY,AAPL,MSFT,AMZN,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,20211021,0.0037,0.0020,-0.0098,0.0,0.0003,0.002608,0.001474,0.010897,0.005842,...,-0.000329,-0.002156,-0.004130,0.010683,0.001944,0.005416,0.009789,0.012832,-0.004696,0.012841
1,20211022,-0.0025,-0.0023,0.0102,0.0,0.0031,-0.001036,-0.005285,-0.005149,-0.028955,...,0.015748,0.005984,-0.000207,-0.002181,0.003732,0.003913,-0.002568,-0.010957,0.013526,-0.008040
2,20211025,0.0058,0.0049,-0.0016,0.0,0.0124,0.005363,-0.000336,-0.003332,-0.004551,...,0.002221,-0.002974,-0.005599,0.011770,0.006395,0.004618,-0.009720,-0.000173,0.002017,0.017145
3,20211026,0.0004,-0.0071,-0.0032,0.0,-0.0022,0.000900,0.004575,0.006426,0.016775,...,-0.002263,0.008121,0.003337,-0.003545,0.000887,-0.118035,0.000476,-0.006752,-0.002013,0.001073
4,20211027,-0.0076,-0.0074,-0.0119,0.0,-0.0008,-0.004430,-0.003148,0.042114,0.004864,...,-0.014625,-0.007233,-0.012679,-0.052368,-0.014174,-0.002922,-0.017132,-0.054210,-0.022191,-0.011786
5,20211028,0.0114,0.0087,-0.0047,0.0,0.0071,0.009649,0.024992,0.003651,0.015941,...,0.000658,0.009273,-0.061474,0.011733,0.008537,0.001360,-0.000223,-0.000369,0.013966,0.009913
6,20211029,0.0022,0.0019,-0.0082,0.0,0.0054,0.002029,-0.018156,0.022414,-0.021511,...,-0.009481,-0.003445,-0.010543,0.005972,-0.036531,0.002806,-0.008829,0.003503,-0.006574,0.004448
7,20211110,-0.0107,-0.0064,0.0095,0.0,-0.0091,-0.008045,-0.019163,-0.015330,-0.026335,...,-0.007873,0.007873,-0.000893,-0.022378,0.017687,-0.001597,-0.014936,0.011088,-0.005758,-0.010333
8,20211111,0.0024,0.0052,0.0032,0.0,0.0101,0.000324,-0.000338,0.004927,-0.002743,...,0.000925,-0.004145,0.003352,0.012385,0.001634,-0.005154,-0.028186,0.043023,-0.007043,-0.010731
9,20211112,0.0074,-0.0033,-0.0090,0.0,-0.0022,0.007547,0.014337,0.012905,0.015162,...,-0.009679,0.004962,0.000445,0.009230,0.000742,0.009438,0.003492,0.025558,-0.011665,0.008502


## Find parameter for each stock

In [38]:
'''
The alpha(intercept) is 0 for this function
First row: parameter value of beta_market for each stock
Second row: parameter value of beta_SMB for each stock
Third row: parameter value of beta_HML for each stock
Fourth row: parameter value of beta_UMD for each stock
'''

def famafrenchParameter(dataframe, stocks):
    m = len(stocks)
    n = len(dataframe)
    
    beta_market = np.zeros(m)
    beta_SMB = np.zeros(m)
    beta_HML = np.zeros(m)
    beta_UMD = np.zeros(m)
    
    for i in range(m):
        y = data_ret[ stocks[i] ] - data_ret["RF"]
        X = data_ret[ ["Mkt-RF", "SMB", "HML", "Mom"] ]
            
        model = sm.OLS(y,X)
        result = model.fit()
            
        beta_market[i] = result.params[0]
        beta_SMB[i] = result.params[1]
        beta_HML[i] = result.params[2]
        beta_UMD[i] = result.params[3]
    
    data = [beta_market, beta_SMB, beta_HML, beta_UMD]
    paramData = pd.DataFrame(data, columns = stocks)
    return paramData

In [39]:
stocks = ["AAPL", "FB", "UNH", "MA", "MSFT", "NVDA", "HD", "PFE", "AMZN", "BRK-B", "PG", "XOM", "TSLA", "JPM", "V", "DIS", "GOOGL", "JNJ", "BAC", "CSCO"]
paramData = famafrenchParameter(data_ret, stocks)
paramData

,AAPL,FB,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
0,0.396370,1.802933,1.041994,2.329145,0.987516,1.396317,0.502289,-1.027977,1.176507,0.721355,0.685326,1.057030,0.201834,0.688310,1.400588,1.197383,1.041799,0.756358,0.963967,1.087121
1,-0.010479,-0.262224,-0.604041,0.234015,-0.691230,-0.651718,-0.634933,-1.034053,-0.299679,-0.288931,-0.866511,0.367565,0.664188,0.374823,0.408646,0.233703,-1.034989,-0.862177,0.313229,-0.470470
2,-0.461813,-0.322528,-0.038723,0.634990,-0.705790,-1.175671,-0.297403,-0.321940,-0.457455,0.642745,0.293486,0.946179,-0.720994,0.997104,0.635346,0.176471,-0.617784,0.101252,1.167663,0.152429
3,1.083024,-0.679598,-0.070917,-1.579253,0.480713,2.093697,0.928475,0.302097,-0.343535,-0.333587,-0.183895,0.085480,2.574986,-0.222833,-0.873302,-0.883878,0.161052,-0.572452,-0.266079,-0.426619


## Calculate daily return for each stock based on 4 factors' parameter

In [40]:
def annualReturn_4factors (factorData, paramData, stocks):
    n = len(factorData)
    m = len(stocks)
    
    for i in range(m):
        stock = stocks[i]
        factorData[stock] = None
        for j in range(n):
            factorData[stock][j] = factorData["RF"][j] + paramData[stock][0]*factorData["Mkt-RF"][j] + paramData[stock][1]*factorData["SMB"][j] + paramData[stock][2]*factorData["HML"][j] + paramData[stock][3]*factorData["Mom"][j]
            
    return factorData

In [41]:
data_dailyReturn = annualReturn_4factors(data, paramData, stocks)

<ipython-input-40-efdb4be0e905>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorData[stock][j] = factorData["RF"][j] + paramData[stock][0]*factorData["Mkt-RF"][j] + paramData[stock][1]*factorData["SMB"][j] + paramData[stock][2]*factorData["HML"][j] + paramData[stock][3]*factorData["Mom"][j]


## Calculate annual return based on daily return (Geometric Returns)

In [43]:
#Add year column in the dataframe
data_dailyReturn["Year"] = None
for i in range( len(data_dailyReturn) ):
    dateformat = parse(str(data_dailyReturn["Date"][i]))
    data_dailyReturn["Year"][i] = dateformat.year
    
data_dailyReturn

<ipython-input-43-94a9efb29219>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dailyReturn["Year"][i] = dateformat.year


,Date,Mkt-RF,SMB,HML,RF,Mom,AAPL,FB,UNH,MA,...,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO,Year
0,19261103,0.0020,-0.0022,-0.0029,0.00013,0.0056,0.00835,0.001442,0.003258,-0.006412,...,-0.00083,0.015583,-0.003457,-0.004701,-0.003451,0.007184,0.00004,-0.003507,0.000508,1926
1,19261104,0.0059,-0.0014,0.0070,0.00013,-0.0050,-0.006165,0.012275,0.007207,0.025886,...,0.012048,-0.017531,0.01176,0.016635,0.012522,0.002596,0.009371,0.014883,0.010403,1926
2,19261105,0.0007,-0.0008,0.0025,0.00013,0.0117,0.011933,-0.007156,0.000416,-0.015317,...,0.003941,0.028065,0.000198,-0.007846,-0.009119,0.002027,-0.005095,0.00036,-0.003343,1926
3,19261106,0.0016,-0.0029,0.0005,0.00013,-0.0003,0.000239,0.003818,0.003551,0.003969,...,0.001203,-0.002606,0.00071,0.001766,0.001721,0.004441,0.004063,0.001428,0.003438,1926
4,19261108,0.0052,-0.0007,0.0008,0.00013,-0.0001,0.001721,0.009499,0.005947,0.012744,...,0.006118,-0.00012,0.004267,0.007723,0.006422,0.005762,0.004805,0.005884,0.006277,1926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25062,20220125,-0.0143,-0.0060,0.0279,0.00000,0.0026,-0.015674,-0.034974,-0.012541,-0.021101,...,0.0093,-0.020292,0.015148,-0.007025,-0.015899,-0.025505,-0.004306,0.016222,-0.009579,2022
25063,20220126,-0.0030,-0.0108,0.0009,0.00000,0.0139,0.013562,-0.012313,0.002377,-0.030895,...,-0.005101,0.027365,-0.008313,-0.020182,-0.018243,0.009735,-0.000824,-0.008922,-0.003973,2022
25064,20220127,-0.0078,-0.0163,0.0077,0.00000,0.0030,-0.003228,-0.014311,0.001207,-0.02183,...,-0.006694,-0.010227,-0.004469,-0.015313,-0.014442,0.004471,0.007216,-0.004432,-0.000917,2022
25065,20220128,0.0245,-0.0015,-0.0207,0.00000,-0.0027,0.016362,0.053076,0.027428,0.047833,...,0.005529,0.011921,-0.003737,0.022908,0.027719,0.03943,0.019274,-0.000305,0.025337,2022


In [86]:
year_list = np.arange(1929, 2023, 1)
year_num = len(year_list)
n = len(data_dailyReturn)
m = len(stocks)

annualReturn = np.zeros((year_num, m))

for i in range(0, year_num):
    year = year_list[i]
    temp = data_dailyReturn[ data_dailyReturn["Year"] == year ]
    temp_len = len(temp)
    for j in range(m):
        stock = stocks[j]
        sqrtGeo = 1
        for k in range(0, temp_len):
            sqrtGeo = sqrtGeo * (1 + temp[stock].iloc[k])
        annualReturn[i][j] = sqrtGeo**(1/temp_len) - 1

annRetdf = pd.DataFrame(data = annualReturn, columns = stocks)
annRetdf #from 1926 to 2022 annual return

,AAPL,FB,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
0,0.001161,-0.002224,0.000096,-0.004325,0.000623,0.001828,0.001782,0.002199,-0.001041,-0.000050,0.000774,-0.000478,0.002110,-0.000614,-0.002367,-0.002189,0.000627,0.000078,-0.000708,-0.000505
1,0.000760,-0.002686,-0.001109,-0.005225,-0.000139,0.000846,0.000566,0.001857,-0.001351,-0.001422,-0.000887,-0.001855,0.001947,-0.001690,-0.003050,-0.002425,-0.000469,-0.001207,-0.002184,-0.001643
2,-0.000083,-0.004150,-0.002285,-0.007237,-0.001389,-0.001421,-0.000529,0.001798,-0.002259,-0.002180,-0.001955,-0.002718,0.000161,-0.002133,-0.003871,-0.003002,-0.001892,-0.002193,-0.002958,-0.002725
3,-0.002287,-0.000511,-0.000479,-0.000196,-0.001613,-0.005094,-0.001893,-0.000955,-0.000245,0.000524,-0.000018,-0.000047,-0.006413,0.000670,0.000806,0.000873,-0.001292,0.000368,0.000545,0.000079
4,0.000174,0.002101,0.001052,0.003539,0.000286,-0.000057,-0.000055,-0.003520,0.001212,0.001461,0.000594,0.002498,-0.000196,0.002211,0.002894,0.002232,0.000131,0.000574,0.002539,0.001415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.000682,-0.000673,-0.000165,-0.001820,0.000396,0.001056,0.000592,0.000668,-0.000206,-0.000534,-0.000225,-0.000644,0.001369,-0.000714,-0.001112,-0.000822,0.000233,-0.000357,-0.000886,-0.000453
90,0.000472,0.001971,0.001224,0.001927,0.001327,0.001498,0.000658,-0.000767,0.001447,0.000649,0.000830,0.000713,-0.000115,0.000356,0.001140,0.001163,0.001452,0.000988,0.000566,0.001202
91,0.000400,0.001542,0.000648,-0.000508,0.001181,0.000155,0.000109,-0.001678,0.001493,-0.000435,-0.000187,-0.000406,-0.001078,-0.000750,-0.000007,0.000685,0.001087,0.000142,-0.000935,0.000465
92,-0.000496,0.001441,0.000923,0.002830,0.000080,-0.000956,-0.000110,-0.001244,0.000724,0.001256,0.000950,0.001549,-0.001884,0.001383,0.001891,0.001390,0.000354,0.000954,0.001755,0.001218


## Annual covariance matrix for 10 stocks

In [91]:
covMatrix = annRetdf.cov()
covMatrix

,AAPL,FB,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,8.234037e-07,4.058016e-07,2.302883e-07,-1.078420e-07,7.570464e-07,1.799198e-06,6.243651e-07,-4.072335e-09,4.014883e-07,-2.538577e-07,-9.821772e-08,-6.943079e-08,1.713135e-06,-3.149155e-07,-1.721887e-07,-4.225127e-08,5.633126e-07,-1.344701e-07,-3.384580e-07,2.957481e-08
FB,4.058016e-07,2.705058e-06,1.131802e-06,3.506890e-06,1.194601e-06,1.504557e-06,2.247278e-07,-1.658683e-06,1.761158e-06,5.704631e-07,4.105976e-07,9.980963e-07,2.188426e-07,5.944864e-07,1.975177e-06,1.867154e-06,1.182032e-06,7.364907e-07,8.564177e-07,1.270305e-06
UNH,2.302883e-07,1.131802e-06,5.613422e-07,1.482163e-06,5.750945e-07,7.912857e-07,2.282020e-07,-5.698142e-07,7.338174e-07,3.276445e-07,3.067662e-07,4.933230e-07,1.981160e-07,2.987430e-07,8.185211e-07,7.420957e-07,5.969979e-07,4.016052e-07,4.364584e-07,5.994145e-07
MA,-1.078420e-07,3.506890e-06,1.482163e-06,5.578949e-06,9.283200e-07,5.798601e-07,-1.610025e-07,-2.404613e-06,2.061690e-06,1.307438e-06,8.238682e-07,1.935633e-06,-9.170704e-07,1.536859e-06,3.290434e-06,2.811072e-06,1.080478e-06,1.224404e-06,2.022067e-06,1.902907e-06
MSFT,7.570464e-07,1.194601e-06,5.750945e-07,9.283200e-07,1.040764e-06,1.917183e-06,5.928619e-07,-4.363117e-07,9.117386e-07,-4.046997e-08,9.967170e-08,1.502869e-07,1.288666e-06,-1.625152e-07,4.016455e-07,5.200940e-07,9.170736e-07,1.930596e-07,-1.074619e-07,4.448611e-07
NVDA,1.799198e-06,1.504557e-06,7.912857e-07,5.798601e-07,1.917183e-06,4.187640e-06,1.400096e-06,-3.203134e-07,1.269933e-06,-3.850004e-07,-4.210819e-08,3.919420e-08,3.563297e-06,-5.708992e-07,5.526170e-08,3.164993e-07,1.542974e-06,-3.928323e-08,-5.503625e-07,3.923873e-07
HD,6.243651e-07,2.247278e-07,2.282020e-07,-1.610025e-07,5.928619e-07,1.400096e-06,5.737534e-07,1.875966e-07,2.436865e-07,-1.112264e-07,6.452474e-08,-2.707622e-08,1.282687e-06,-2.133387e-07,-1.982563e-07,-1.242222e-07,4.919448e-07,-5.373066e-09,-2.099905e-07,6.035515e-08
PFE,-4.072335e-09,-1.658683e-06,-5.698142e-07,-2.404613e-06,-4.363117e-07,-3.203134e-07,1.875966e-07,1.436851e-06,-1.017370e-06,-3.945622e-07,-1.121598e-07,-7.732549e-07,3.202785e-07,-5.734175e-07,-1.471743e-06,-1.329806e-06,-4.098438e-07,-3.441812e-07,-7.406677e-07,-7.499598e-07
AMZN,4.014883e-07,1.761158e-06,7.338174e-07,2.061690e-06,9.117386e-07,1.269933e-06,2.436865e-07,-1.017370e-06,1.196466e-06,2.502089e-07,2.053804e-07,5.100325e-07,3.987973e-07,2.225803e-07,1.130744e-06,1.134940e-06,8.671737e-07,4.239452e-07,3.607481e-07,7.728774e-07
BRK-B,-2.538577e-07,5.704631e-07,3.276445e-07,1.307438e-06,-4.046997e-08,-3.850004e-07,-1.112264e-07,-3.945622e-07,2.502089e-07,5.242409e-07,3.798803e-07,6.234109e-07,-6.542939e-07,5.952660e-07,8.162750e-07,5.853822e-07,8.698594e-08,4.076289e-07,7.537456e-07,4.795645e-07


## Super efficient portfolio

In [92]:
#set weight for each stocks
weights = np.ones(20)*(1/20)

#initial some important value
meanReturns = annRetdf.mean()

In [95]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

In [96]:
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

In [97]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

In [98]:
def maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [101]:
riskFreeRate = 0.0025
result = maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet=(0,1))
maxSR, maxWeights = result['fun'], result['x']

-12.555078247367922 [2.14975840e-01 3.15479644e-13 3.54330687e-14 1.00035807e-12
 0.00000000e+00 0.00000000e+00 2.30151003e-01 7.25823206e-02
 3.77216975e-14 1.24673567e-13 9.17381541e-14 0.00000000e+00
 0.00000000e+00 4.82290836e-01 3.76476939e-13 2.48764815e-13
 0.00000000e+00 1.67214285e-13 0.00000000e+00 1.48520376e-13]


In [102]:
print("The maximum sharp ratio is: %f" % maxSR)
print("The weights for each stock should be:")
print(maxWeights)

The maximum sharp ratio is: -12.555078
The weights for each stock should be:
[2.14975840e-01 3.15479644e-13 3.54330687e-14 1.00035807e-12
 0.00000000e+00 0.00000000e+00 2.30151003e-01 7.25823206e-02
 3.77216975e-14 1.24673567e-13 9.17381541e-14 0.00000000e+00
 0.00000000e+00 4.82290836e-01 3.76476939e-13 2.48764815e-13
 0.00000000e+00 1.67214285e-13 0.00000000e+00 1.48520376e-13]
